In [215]:
# update_db.py
import os
import pandas as pd
import sqlalchemy as sql
from sqlalchemy.sql.schema import Column
from sqlalchemy.sql.sqltypes import String

# print("file path:", os.getcwd()) # root should in ./tdtoolkit_web
raw_root = os.path.join('.','src','app', 'raw')

path = {
    "axo": os.path.join(raw_root, "AXO"),
    "rdl": os.path.join(raw_root, "RDL"),
    "opt": os.path.join(raw_root, "OPT"),
    "rt": os.path.join(raw_root, "RT"),
    "cond": os.path.join(raw_root, "CONDITIONS"),
    "prop": os.path.join(raw_root, "PROPERTY"),
    "ref": os.path.join(raw_root, "REF"),
    "output": "output",
    "db": "database"
}

# need to separate later
def axo_load(path, cond=pd.DataFrame()):
    """
    The path and the file name need to be below:
    Could be multiple file
    ─┬─ folder[path]
     ├─ 1-1 + 1-2 + 1-3.csv
     ├─ 1-4.csv
     ...
    """
    df = pd.DataFrame()
    # may wrong, need caution
    loc = [5, 3, 1, 6, 4, 2]
    for cwd, dir_name, file_names in os.walk(path):
        for f in file_names:
            file = os.path.join(cwd, f)
            # get short-id from file name
            # Cause there may be multiple panel in one file, we need to
            # deal with it.
            short_id = f.split(".")[0]
            short_id = [s.strip() for s in short_id.split("+")]
            # multiple each point 6 times cause there are 6 point in one panel
            # maybe I should split this function to another place?
            short_id_6 = [id for id in short_id for _ in range(6)]
            location_6 = loc * len(short_id)
            # Todo: more error format handling
            
            tmp_df = pd.read_csv(file, engine="python", skiprows=27, skipfooter=92)
            tmp_df.insert(loc = 1, column = "short-id", value=short_id_6)
            tmp_df.insert(loc = 2, column = "point", value = location_6)
            # some data has different title(?), so we rename it to make concat well 
            tmp_df.columns = ["Chip No.", "Short-id",  "Point", "x", "y", "cell gap", "top rubbing direct", "twist", "top pre-tilt", "bottom pre-tilt", "rms", "iteration"]
            df = pd.concat([df, tmp_df], ignore_index=True)
    # replace short-id if you have condition table
    if cond.empty != True:
        df = df.rename(columns={"Short-id": "ID", "Chip No.": "LC"})
        df["ID"] = df["ID"].map(dict(cond[["Short-id", "ID"]].values))
        df["LC"] = df["ID"].map(dict(cond[["ID", "LC"]].values))
        df["Project"] = df["ID"].map(dict(cond[["ID", "Project"]].values))
        df["Batch"] = df["ID"].map(dict(cond[["ID", "Batch"]].values))
        # neglect the data that doesn't record
        df = df[~df["ID"].isna()]
        df.columns = ["LC", "ID", "Point", "x", "y", "cell gap", "top rubbing direct", "twist", "top pre-tilt", "bottom pre-tilt", "rms", "iteration", "Project", "Batch"]

    return df

def rdl_load(path, cond=pd.DataFrame()):
    """
    The path and the file name need to be below:
    Should be single file.
    ─┬─ folder[path]
     └─ [cell gap].xlsx
    """
    file = next(os.walk(path))[2][0]
    df = pd.read_excel(os.path.join(path, file))
    if cond.empty != True:
        df = df.rename(columns={"Short-id": "ID"})
        df["ID"] = df["ID"].map(dict(cond[["Short-id", "ID"]].values))
        df["LC"] = df["ID"].map(dict(cond[["ID", "LC"]].values))
        df["Project"] = df["ID"].map(dict(cond[["ID", "Project"]].values))
        df["Batch"] = df["ID"].map(dict(cond[["ID", "Batch"]].values))
        # neglect the data that doesn't record
        df = df[~df["ID"].isna()]
        df.columns = ['ID', 'cell gap', 'LC', "Project", "Batch"]
    return df

def opt_load(path, cond=pd.DataFrame()):
    """
    The path and the file name need to be below:
    Could be multiple file
    ─┬─ folder[path]
     ├─ xxxx.csv
     ├─ xxxx.csv
     ...
    """
    df = pd.DataFrame()
    for cwd, dir_name, file_names in os.walk(path):
        for f in file_names:
            if f[0] == ".":
                continue
            if f[0] == "~":
                continue
            file = os.path.join(cwd, f)
            tmp_df = pd.read_csv(file, encoding="ansi").iloc[:,:64]
            # some data preprocessing
            df = pd.concat([df, tmp_df], ignore_index=True)
            
    df.columns = ['Data', 'M_Time', 'ID', 'Point', 'Station', 'Operator', 'Voltage',
        'I.Time', 'AR_T%(⊥)', 'AR_T%(//)', 'LCM_X%', 'LCM_Y%', 'LCM_Z%', 'RX',
        'RY', 'RZ', 'GX', 'GY', 'GZ', 'BX', 'BY', 'BZ', 'WX', 'WY', 'WZ', 'CG%',
        'R_x', 'R_y', 'G_x', 'G_y', 'B_x', 'B_y', 'W_x', 'W_y', 'RX_max',
        'GY_max', 'BZ_max', 'V_RX_max', 'V_GY_max', 'V_BZ_max', "WX'", "WY'",
        "WZ'", "W_x'"," W_x'.1", 'LCM_X%max', 'LCM_Y%max', 'LCM_Z%max',
        'φ_(Ymax)', 'φ_(Ymax).1', 'φ_(Zmax)', 'φ_tol_X', 'φ_tol_Y', 'φ_tol_Z',
        'T0/Tmax_X', 'T0/Tmax_Y', 'T0/Tmax_Z', 'Vcri_X', 'Vcri_Y', 'Vcri_Z',
        'dφ_X', 'dφ_Y', 'dφ_Z', "LC"]
    # voltage == 1 is the wrong rows, need drop
    df = df[df["Voltage"] != 1]
    if cond.empty != True:
        df["LC"] = df["ID"].map(dict(cond[["ID", "LC"]].values))
        df["Project"] = df["ID"].map(dict(cond[["ID", "Project"]].values))
        df["Batch"] = df["ID"].map(dict(cond[["ID", "Batch"]].values))
    return df

def rt_load(path, cond=pd.DataFrame()):
    """
    The path and the file name need to be below:
    Could be multiple file
    ─┬─ folder[path]
     ├─ xxxx.txt
     ├─ xxxx.txt
     ...
    """
    df = pd.DataFrame()
    for cwd, dir_name, file_names in os.walk(path):
        for f in file_names:
            if f[0] == ".":
                continue
            if f[0] == "~":
                continue
            file = os.path.join(cwd, f)
            tmp_df = pd.read_table(file, encoding="ansi")
            # some system encoding would go wrong, so I rename here
            # there are some implicit problem, the data should just like this
            tmp_df.columns = ['Date', 'Time', 'ID', 'Point', 'Station', 'Operator', 'cell pos.', 'Target Vpk',
               'Initial Vpk', 'OD_Rise', 'OD_fall', 'Normalized_V', 'Specific_target',
               'Photo Sensor', 'TempSensor', 'Temp', 'Model', 'Rise-mean (10-90)',
               'Rise-stdev (10-90)', 'Fall-mean (10-90)', 'Fall-stdev (10-90)',
               'Rise-mean (5-95)', 'Rise-stdev (5-95)', 'Fall-mean (5-95)',
               'Fall-stdev (5-95)', 'Vcom', 'Flicker', 'Base lv-mean', 'Top lv-mean',
               'WXT (%)', 'BXT (%)', 'WXT_*', 'BXT_*', 'Overshooting or not',
               'Overshooting %', 'TailTime', 'overshooting_peak', 'overshooting_top',
               '(RisePeak-top)/top', '(FallPeak-base)/base', 'delta_peak', 'delta_v',
               'delta_m', 'c_a', 'peak', 'top', 'HLH_(Peak-Top)', 'HLH_area']
            # neglect incorrect rows
            tmp_df = tmp_df[tmp_df["Point"].isin([1, 2, 3, 4, 5, 6, '1', '2', '3', '4', '5', '6'])]
            # correct the data types
            for col in tmp_df.columns:
                try:
                    tmp_df[col] = tmp_df[col].astype('float')
                except:
                    continue
            df = pd.concat([df, tmp_df], ignore_index=True)
    
    if cond.empty != True:
        df["LC"] = df["ID"].map(dict(cond[["ID", "LC"]].values))
        df["Project"] = df["ID"].map(dict(cond[["ID", "Project"]].values))
        df["Batch"] = df["ID"].map(dict(cond[["ID", "Batch"]].values))
    df = df[df["ID"]!="NAN"]
    
    return df

def cond_load(path):
    """
    The path and the file name need to be below:
    Should be single file.
    ─┬─ folder[path]
     └─ [cond].xlsx
    """
    file = next(os.walk(path))[2][0]
    df = pd.read_excel(os.path.join(path, file))
    df = df.iloc[:,0:5]
    df.columns = ["ID", "LC", "Short-id", "Project", "Batch"]
    return df

def prop_load(path):
    """
    The path and the file name need to be below:
    Should be single file.
    ─┬─ folder[path]
     └─ [prop].xlsx
    """
    file = next(os.walk(path))[2][0]
    df = pd.read_excel(os.path.join(path, file))
    df['Scatter index'] = (df['n_e'] ** 2 - df['n_o'] ** 2) ** 2 * 3 / (df['K11(pN)'] + df['K22(pN)'] + df['K33(pN)'])
    df['RT index'] = df['rotation viscosity (γ1)(mPa⋅s)'] / df['K22(pN)']
    return df

def ref_load(path):
    """
    The path and the file name need to be below:
    Should be single file.
    ─┬─ folder[path]
     └─ [ref].xlsx
    """
    file = next(os.walk(path))[2][0]
    df = pd.read_excel(os.path.join(path, file))
    return df

# loading data from raw
cond = cond_load(path["cond"])
axo = axo_load(path["axo"], cond)
rdl = rdl_load(path["rdl"], cond)
opt = opt_load(path["opt"], cond)
rt = rt_load(path["rt"], cond)
prop = prop_load(path["prop"])
ref = ref_load(path["ref"])


# writing to database

engine = sql.create_engine('sqlite:///database/test.db', echo=False)
# engine = sql.create_engine('sqlite://', echo=True)

meta = sql.MetaData()
# Model 是否要分出去?
sql.Table(
    "cond", meta,
    sql.Column("LC", sql.String),
    sql.Column("Short-id", sql.String),
    sql.Column("ID", sql.String, unique=True),
    sql.Column("Project", sql.String),
    sql.Column("Batch", sql.String)
)

meta.create_all(engine)

## check constrain
try:
    cond.to_sql("cond", con=engine, if_exists="append", index=False)
    axo.to_sql("axo", con=engine, if_exists="append", index=False)
    rdl.to_sql("rdl", con=engine, if_exists="append", index=False)
    opt.to_sql("opt", con=engine, if_exists="append", index=False)
    rt.to_sql("rt", con=engine, if_exists="append", index=False)
    prop.to_sql("prop", con=engine, if_exists="append", index=False)
    ref.to_sql("ref", con=engine, if_exists="append", index=False)
    print("Database update!")
except:
    print("Something wrong, maybe the ID condition is duplicate. Database keep")

Database update!


In [ ]:
# calculate_summary.py

In [2]:
import sys
import os
import numpy as np
import pandas as pd
import sqlalchemy as sql
from scipy.interpolate import interp1d
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn import linear_model
# from xgboost import XGBRegressor

In [216]:
# batch = "RD11006008" # X50
batch = "RD11001105" # 1098 TR2

engine = sql.create_engine('sqlite:///database/test.db', echo=False)

cond = pd.read_sql(f"SELECT * FROM cond WHERE batch == \"{batch}\"", engine)
axo = pd.read_sql(f"SELECT * FROM axo WHERE batch == \"{batch}\"", engine)
rdl = pd.read_sql(f"SELECT * FROM rdl WHERE batch == \"{batch}\"", engine)
opt = pd.read_sql(f"SELECT * FROM opt WHERE batch == \"{batch}\"", engine)
rt = pd.read_sql(f"SELECT * FROM rt WHERE batch == \"{batch}\"", engine)
prop = pd.read_sql(f"SELECT * FROM prop", engine)
ref = pd.read_sql(f"SELECT * FROM ref WHERE batch == \"{batch}\"", engine)


In [217]:
prop

,LC,Vendor,measure T(°C),Tni(°C),Tcn(°C),design cell gap,n_e,n_o,ε_∥,ε_⊥,rotation viscosity (γ1)(mPa⋅s),K11(pN),K22(pN),K33(pN),Scatter index,RT index
0,LCT-15-1098,Merck,25,85,None,3.1,1.577,1.480,3.6,7.0,83,14.7,7.35,15.1,0.007101,11.292517
1,SLC19V33L00,SLC,25,80,None,3.1,1.603,1.488,3.9,8.1,97,17.0,8.50,15.0,0.009360,11.411765
2,SLC20V87L00,SLC,25,80,None,3.1,1.587,1.484,3.6,6.9,76,14.6,7.30,14.8,0.008179,10.410959
3,AV369-031-TA,HCCH,25,80,None,3.1,1.590,1.486,3.5,6.9,53,15.4,7.70,17.0,0.007656,6.883117
4,ZIX-7054XX,JNC,25,80,None,3.1,1.589,1.486,3.5,6.6,88,13.5,6.75,14.8,0.008586,13.037037


In [8]:
cond.LC.unique()

array(['LCT-15-1098', 'SLC19V33L00', 'SLC20V87L00', 'AV369-031-TA',
       'ZIX-7054XX'], dtype=object)

In [9]:
def custom_f(X):
    features = np.empty(shape=(len(X), 5), dtype=float)
    features[:, 0] = 1
    features[:, 1] = X[:, 0]
    features[:, 2] = X[:, 1]
    features[:, 3] = X[:, 0] * X[:, 1]
    features[:, 4] = X[:, 0] ** 2
    return features
transformer = FunctionTransformer(custom_f)

In [10]:
ref_Tr = ref["Tr(ms)"][0]
ref_cell_gap = ref["cell gap(um)"][0]
ref_LC = ref["LC"][0]

# check is there axo data
if len(axo) != 0:
    rt_cell_gap = pd.merge(rt, axo[["ID", "Point", "cell gap"]], how="left", on=["ID", "Point"])
else:
    rt_cell_gap = pd.merge(rt, rdl[["ID", "cell gap"]], how="left", on="ID")
    
df = rt_cell_gap[rt_cell_gap["LC"] == ref_LC].copy()
df["Tr"] = df["Rise-mean (10-90)"]
df["Vop"] = df["Target Vpk"]

df = df.groupby(by=["ID", "Vop", "Point"], as_index=False).mean()

# sns.scatterplot(data=df, x="Vop", y="Tr")

model = {}
# Let's try some fasion ML (XD
training_set, test_set = train_test_split(
    df,
    test_size = 0.2,
    random_state = 42
)
X_train = training_set[["Tr", "cell gap"]].to_numpy()
y_train = training_set["Vop"].to_numpy()
X_test = test_set[["Tr", "cell gap"]].to_numpy()
y_test = test_set["Vop"].to_numpy()
valid_data = [[ref_Tr, ref_cell_gap]]

# # eXtreme Grandient Boostng Regression
# # -> Although it can easily get high R2_score, Hard to get physics trend.
# model["Vop_ref_XGBR"] = XGBRegressor(
#     n_estimators = 50,
#     learning_rate = 0.1,
#     max_depth = 3,
#     gamma = 0.01,
#     reg_lambda = 0.01
# )
# model["Vop_ref_XGBR"].fit(
#     X_train, y_train,
#     early_stopping_rounds = 10,
#     eval_set = [(X_test, y_test)],
#     verbose = False
# )

# Linear regression

model["Vop_ref_LR"] = Pipeline([
    ('Scalar', StandardScaler()),
    ('poly', PolynomialFeatures(degree=2)),
    ('linear', linear_model.LinearRegression(fit_intercept=False))]
).fit(
    X_train, y_train,
)

print("R2_train:", model["Vop_ref_LR"].score(X_train, y_train))
print("R2_test:", model["Vop_ref_LR"].score(X_test, y_test))
ref_Vop = float(model["Vop_ref_LR"].predict(valid_data))
print("Vop from Ref[Tr, cell gap]:", ref_Vop)

# Calculate RT, Tf, Tr
df = rt_cell_gap.copy()
df["Vop"] = df["Target Vpk"]
df["RT"] = df["Rise-mean (10-90)"] + df["Fall-mean (10-90)"]
df["Tr"] = df["Rise-mean (10-90)"]
df["Tf"] = df["Fall-mean (10-90)"]
training_set, test_set = train_test_split(
    df,
    test_size = 0.1,
)

model["rt"] = {}

for LC in cond["LC"].unique():
    print(LC)
    model["rt"][LC] = {}
    X_train = training_set[training_set["LC"]==LC][["Vop", "cell gap"]].to_numpy()
    X_test = test_set[test_set["LC"]==LC][["Vop", "cell gap"]].to_numpy()
    valid_data = [[ref_Vop, ref_cell_gap]]
    
    for item in ["Tr", "Tf", "RT"]:
        y_train = training_set[training_set["LC"]==LC][item].to_numpy()
        y_test = test_set[test_set["LC"]==LC][item].to_numpy()

        model["rt"][LC][f"{item}_LR"] = Pipeline([
            ('Scalar', StandardScaler()),
#             ('poly', PolynomialFeatures(degree=1)),
            ('Custom_Transformer', transformer),
            ('linear', linear_model.TheilSenRegressor(fit_intercept=False))
        ]).fit(
            X_train, y_train,
        )
        print(f'R2_test {model["rt"][LC][f"{item}_LR"].score(X_test, y_test):.2f}')
        ans = float(model["rt"][LC][f"{item}_LR"].predict(valid_data))
        print(f"{LC}: {item}: {ans:.2f} ms")
        print()


R2_train: 0.982181877589628
R2_test: 0.9808685313384988
Vop from Ref[Tr, cell gap]: 3.8640432213475027
LCT-15-1098
R2_test 0.94
LCT-15-1098: Tr: 15.28 ms

R2_test 0.75
LCT-15-1098: Tf: 10.90 ms

R2_test 0.90
LCT-15-1098: RT: 26.15 ms

SLC19V33L00
R2_test 0.93
SLC19V33L00: Tr: 15.40 ms

R2_test 0.77
SLC19V33L00: Tf: 12.68 ms

R2_test 0.87
SLC19V33L00: RT: 28.15 ms

SLC20V87L00
R2_test 0.94
SLC20V87L00: Tr: 15.55 ms

R2_test -0.06
SLC20V87L00: Tf: 11.29 ms

R2_test 0.86
SLC20V87L00: RT: 26.85 ms

AV369-031-TA
R2_test 0.96
AV369-031-TA: Tr: 15.20 ms

R2_test 0.60
AV369-031-TA: Tf: 10.83 ms

R2_test 0.90
AV369-031-TA: RT: 26.14 ms

ZIX-7054XX
R2_test 0.95
ZIX-7054XX: Tr: 15.32 ms

R2_test 0.45
ZIX-7054XX: Tf: 11.03 ms

R2_test 0.82
ZIX-7054XX: RT: 26.39 ms



In [11]:
# plot for checking
# maybe plot_RT.py?
%matplotlib widget
import matplotlib.pyplot as plt
# ref Vop
# raw
xlabel = "Tr"
ylabel = "cell gap"
zlabel = "Vop"
# plt.figure(figsize=(10,8))
ax = plt.axes(projection="3d")
plot_raw = df[df["LC"] == ref_LC]
ax.scatter(plot_raw[xlabel],plot_raw[ylabel],plot_raw[zlabel], label='raw')
# fitting
x_range = np.linspace(plot_raw[xlabel].min(), plot_raw[xlabel].max(), 50)
y_range = np.linspace(plot_raw[ylabel].min(), plot_raw[ylabel].max(), 50)
x_range, y_range = np.meshgrid(x_range, y_range)
predict_region = np.array(list(zip(x_range.flatten(), y_range.flatten())))
z_predict = model['Vop_ref_LR'].predict(predict_region)
ax.scatter(x_range, y_range, z_predict, label="fitting surface", alpha=0.1)
plt.title(ref_LC)
plt.legend
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.set_zlabel(zlabel)
plt.show()

# sample Tr
xlabel = "Vop"
ylabel = "cell gap"
zlabel = "Tr"
for LC in cond["LC"].unique():
    plt.figure(figsize=(5,4))
    ax = plt.axes(projection="3d")
    plot_raw = df[df["LC"] == LC]
    ax.scatter(plot_raw[xlabel],plot_raw[ylabel],plot_raw[zlabel], label=LC)
    # fitting
    x_range = np.linspace(plot_raw[xlabel].min()-0.1, plot_raw[xlabel].max()+0.1, 50)
    y_range = np.linspace(plot_raw[ylabel].min()-0.1, plot_raw[ylabel].max()+0.1, 50)
    x_range, y_range = np.meshgrid(x_range, y_range)
    predict_region = np.array(list(zip(x_range.flatten(), y_range.flatten())))
    z_predict = model["rt"][LC][f"Tr_LR"].predict(predict_region)
    ax.scatter(x_range, y_range, z_predict, label="fitting surface", alpha=0.1)
    plt.title(LC)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_zlabel(zlabel)
    plt.show()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [143]:
from scipy.interpolate import interp1d
data = opt[(opt.ID=="T19BR001NL2X") & (opt.Point ==1)]
x = data["Voltage"]
y = data["LCM_Y%"]
f = interp1d(x, y, kind='cubic')
x_dense = np.linspace(0, 20, 1000)
y_dense = f(x_dense)

plt.figure(figsize=(5,4))
plt.plot(x_dense, y_dense, )


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [28]:
def opt_features_extract(X):
    features = np.empty(shape=(len(X), 7), dtype=float)
    features[:, 0] = 1
    features[:, 1] = X[:, 0]
    features[:, 2] = X[:, 1]
    features[:, 3] = X[:, 0] * X[:, 1]
    features[:, 4] = X[:, 0] ** 2
    features[:, 5] = X[:, 0] ** 3
    features[:, 6] = X[:, 0] ** 4

    return features
transformer_opt = FunctionTransformer(opt_features_extract)

In [59]:
def Vop_features_extract(X):
    # Vop = a * exp(T%+10) + b * cell_gap + c
    features = np.empty(shape=(len(X), 3), dtype=float)
    features[:, 0] = 1
#     features[:, 1] = X[:, 0]
    features[:, 1] = X[:, 1]
#     features[:, 3] = X[:, 0] * X[:, 1]
#     features[:, 4] = X[:, 0] ** 2
    features[:, 2] = np.exp(X[:, 0]+10)

    return features
transformer_Vop = FunctionTransformer(Vop_features_extract)

In [21]:
# test V-T max cut-off
test_df = opt[(opt.ID == "T19BR001NL2X") & (opt.Point == 1)]
test_df = test_df.iloc[:test_df["LCM_Y%"].argmax(),:]
plt.figure(figsize=(5,4))
plt.plot(test_df["Voltage"], test_df["LCM_Y%"], '*')
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [22]:
# make a cut off opt data
opt_cutoff = pd.DataFrame(columns=opt.columns)
for ID in opt.ID.unique():
    for Point in opt.Point.unique():
        tmp_df = opt[(opt.ID == ID) & (opt.Point == Point)]
        tmp_df = tmp_df.iloc[:tmp_df["LCM_Y%"].argmax(),:]
        opt_cutoff = pd.concat([opt_cutoff, tmp_df])

In [133]:
opt.columns

Index(['Data', 'M_Time', 'ID', 'Point', 'Station', 'Operator', 'Voltage',
       'I.Time', 'AR_T%(⊥)', 'AR_T%(//)', 'LCM_X%', 'LCM_Y%', 'LCM_Z%', 'RX',
       'RY', 'RZ', 'GX', 'GY', 'GZ', 'BX', 'BY', 'BZ', 'WX', 'WY', 'WZ', 'CG%',
       'R_x', 'R_y', 'G_x', 'G_y', 'B_x', 'B_y', 'W_x', 'W_y', 'RX_max',
       'GY_max', 'BZ_max', 'V_RX_max', 'V_GY_max', 'V_BZ_max', 'WX'', 'WY'',
       'WZ'', 'W_x'', ' W_x'.1', 'LCM_X%max', 'LCM_Y%max', 'LCM_Z%max',
       'φ_(Ymax)', 'φ_(Ymax).1', 'φ_(Zmax)', 'φ_tol_X', 'φ_tol_Y', 'φ_tol_Z',
       'T0/Tmax_X', 'T0/Tmax_Y', 'T0/Tmax_Z', 'Vcri_X', 'Vcri_Y', 'Vcri_Z',
       'dφ_X', 'dφ_Y', 'dφ_Z', 'LC', 'Project', 'Batch'],
      dtype='object')

In [153]:
opt['WZ']

0          0.3059
1          0.4703
2          0.6329
3          0.6374
4          1.0187
           ...   
14989    888.4389
14990    884.5398
14991    878.9590
14992    877.9019
14993    870.2999
Name: WZ, Length: 14994, dtype: float64

In [139]:
# check is there axo data
if len(axo) != 0:
    opt_cell_gap = pd.merge(opt_cutoff, axo[["ID", "Point", "cell gap"]], how="left", on=["ID", "Point"])
else:
    opt_cell_gap = pd.merge(opt_cutoff, rdl[["ID", "cell gap"]], how="left", on="ID")

model["opt"] = {}
df = opt_cell_gap.copy()
# some mapping and rename
df["T%"] = opt_cell_gap.groupby(by=["ID", "Point"])["LCM_Y%"].apply(lambda x: 100*x / float(x.max()))
df["Vop"] = df["Voltage"]/2.0
df["LC%"] = df["LCM_Y%"]
df["Wx"] = df["W_x"]
df["Wy"] = df["W_y"]
# the varient is large when Vop is low, so I cut-off at Vop = 2
df = df[df["Vop"] > 3]
training_set, test_set = train_test_split(
    df,
    test_size = 0.2,
#     random_state = 42
)

for LC in cond["LC"].unique():
    model["opt"][LC] = {}
    X_train = training_set[training_set["LC"]==LC][["Vop", "cell gap"]].to_numpy()
    X_test = test_set[test_set["LC"]==LC][["Vop", "cell gap"]].to_numpy()
    valid_data = [[ref_Vop, ref_cell_gap]]
    for item in ["T%", "LC%"]:
        y_train = training_set[training_set["LC"]==LC][item].to_numpy()
        y_test = test_set[test_set["LC"]==LC][item].to_numpy()
        model["opt"][LC][f'{item}_LR'] = Pipeline([
            ('Scalar', StandardScaler()),
#             ('poly', PolynomialFeatures(degree=3)),
            ('Custom_Transformer', transformer_opt),
            ('linear', linear_model.TheilSenRegressor(fit_intercept=False)),
#             ('linear', linear_model.LinearRegression(fit_intercept=False)),
#             ("GR", GaussianProcessRegressor(kernel=DotProduct()+WhiteKernel()))
        ]).fit(
            X_train, y_train,
        )
        print(f'R2_test {model["opt"][LC][f"{item}_LR"].score(X_test, y_test):.2f}')
        ans = float(model["opt"][LC][f"{item}_LR"].predict(valid_data))
        print(f"{LC}: {item}: {ans:.4f}")
        print()
    for item in ["Wx", "Wy", "WX", "WY", "WZ"]:
        y_train = training_set[training_set["LC"]==LC][item].to_numpy()
        y_test = test_set[test_set["LC"]==LC][item].to_numpy()
        model["opt"][LC][f'{item}_LR'] = Pipeline([
            ('Scalar', StandardScaler()),
            ('poly', PolynomialFeatures(degree=2)),
#             ('Custom_Transformer', transformer),
            ('linear', linear_model.TheilSenRegressor(fit_intercept=False))
        ]).fit(
            X_train, y_train,
        )
        print(f'R2_test {model["opt"][LC][f"{item}_LR"].score(X_test, y_test):.2f}')
        ans = float(model["opt"][LC][f"{item}_LR"].predict(valid_data))
        print(f"{LC}: {item}: {ans:.4f}")
        print()

# find V%
# f(T%, cell_gap) -> V%
valid_data = [[90.0, ref_cell_gap]]
training_set_Vop = training_set[training_set["T%"]>89]
test_set_Vop = test_set[test_set["T%"]>89]

for LC in cond["LC"].unique():
    X_train = training_set_Vop[training_set_Vop["LC"]==LC][["T%", "cell gap"]].to_numpy()
    X_test = test_set_Vop[test_set_Vop["LC"]==LC][["T%", "cell gap"]].to_numpy()
    y_train = training_set_Vop[training_set_Vop["LC"]==LC]["Vop"].to_numpy()
    y_test = test_set_Vop[test_set_Vop["LC"]==LC]["Vop"].to_numpy()
    model["opt"][LC][f'Vop_LR'] = Pipeline([
        ('Scalar', StandardScaler()),
#         ('poly', PolynomialFeatures(degree=6)),
        ('Custom_Transformer', transformer_Vop),
        ('linear', linear_model.TheilSenRegressor(fit_intercept=False)),
#         ('linear', linear_model.Linsor(kernel=DotProduct()+WhiteKernel()))
    ]).fit(
        X_train, y_train,
    )
    print(f'R2_test {model["opt"][LC][f"Vop_LR"].score(X_test, y_test):.2f}')
    ans = float(model["opt"][LC][f"Vop_LR"].predict(valid_data))
    print(f"{LC}: Vop: {ans:.4f}")
    print()

R2_test 1.00
LCT-15-1098: T%: 78.1463

R2_test 0.99
LCT-15-1098: LC%: 0.6106

R2_test 0.96
LCT-15-1098: Wx: 0.2778

R2_test 0.96
LCT-15-1098: Wy: 0.2791

R2_test 0.97
LCT-15-1098: WX: 477.7983

R2_test 0.97
LCT-15-1098: WY: 479.4125

R2_test 0.96
LCT-15-1098: WZ: 760.4654

R2_test 1.00
SLC19V33L00: T%: 84.1051

R2_test 1.00
SLC19V33L00: LC%: 0.6925

R2_test 0.96
SLC19V33L00: Wx: 0.2998

R2_test 0.96
SLC19V33L00: Wy: 0.3093

R2_test 0.98
SLC19V33L00: WX: 520.5143

R2_test 0.98
SLC19V33L00: WY: 537.9014

R2_test 0.97
SLC19V33L00: WZ: 680.9604

R2_test 1.00
SLC20V87L00: T%: 78.8159

R2_test 1.00
SLC20V87L00: LC%: 0.6424

R2_test 0.70
SLC20V87L00: Wx: 0.2880

R2_test 0.74
SLC20V87L00: Wy: 0.2939

R2_test 0.97
SLC20V87L00: WX: 551.7919

R2_test 0.97
SLC20V87L00: WY: 560.6187

R2_test 0.97
SLC20V87L00: WZ: 801.4759

R2_test 1.00
AV369-031-TA: T%: 77.4233

R2_test 1.00
AV369-031-TA: LC%: 0.6158

R2_test 0.89
AV369-031-TA: Wx: 0.2841

R2_test 0.90
AV369-031-TA: Wy: 0.2881

R2_test 0.97
AV369-0

R2_test 0.94
LCT-15-1098: Vop: 4.6772

R2_test 0.97
SLC19V33L00: Vop: 4.2801

R2_test 0.97
SLC20V87L00: Vop: 4.6623

R2_test 0.97
AV369-031-TA: Vop: 4.7156

R2_test 0.96
ZIX-7054XX: Vop: 4.6041



In [64]:
# plot for checking
# maybe plot_RT.py?
%matplotlib widget
import matplotlib.pyplot as plt

# sample Tr
xlabel = "Vop"
ylabel = "cell gap"
for item in ["T%", "LC%", "Wx", "Wy"]:
    zlabel = item
#     for LC in cond["LC"].unique():
    for LC in ["LCT-15-1098"]:
        plt.figure(figsize=(5,4))
        ax = plt.axes(projection="3d")
        plot_raw = df[df["LC"] == LC]
        ax.scatter(plot_raw[xlabel],plot_raw[ylabel],plot_raw[zlabel], label=LC)
        # fitting
        x_range = np.linspace(plot_raw[xlabel].min()-0.1, plot_raw[xlabel].max()+0.1, 50)
        y_range = np.linspace(plot_raw[ylabel].min()-0.1, plot_raw[ylabel].max()+0.1, 50)
        x_range, y_range = np.meshgrid(x_range, y_range)
        predict_region = np.array(list(zip(x_range.flatten(), y_range.flatten())))
        z_predict = model["opt"][LC][f"{item}_LR"].predict(predict_region)
        ax.scatter(x_range, y_range, z_predict, label="fitting surface", alpha=0.1)
        plt.title(LC)
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.set_zlabel(zlabel)
        plt.show()

plot_raw = plot_raw[plot_raw["T%"]>89]
xlabel = "T%"
ylabel = "cell gap"
zlabel = "Vop"
plt.figure(figsize=(5,4))
ax = plt.axes(projection="3d")
plot_raw = df[df["LC"] == LC]
ax.scatter(plot_raw[xlabel],plot_raw[ylabel],plot_raw[zlabel], label=LC)
# fitting
x_range = np.linspace(plot_raw[xlabel].min()-0.1, plot_raw[xlabel].max()+0.1, 50)
y_range = np.linspace(plot_raw[ylabel].min()-0.1, plot_raw[ylabel].max()+0.1, 50)
x_range, y_range = np.meshgrid(x_range, y_range)
predict_region = np.array(list(zip(x_range.flatten(), y_range.flatten())))
z_predict = model["opt"][LC][f"Vop_LR"].predict(predict_region)
ax.scatter(x_range, y_range, z_predict, label="fitting surface", alpha=0.1)
plt.title(LC)
ax.set_xlabel(xlabel)
ax.set_ylabel(ylabel)
ax.set_zlabel(zlabel)
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [227]:
# Generate table
summary_table = pd.DataFrame(
    columns=["LC", "V90", "V95", "V99", "Vmax", "Vop(V)", "Δnd(nm)", "Gap(um)", "LC%", "Wx", "Wx_gain", "Wy", "Wy_gain",
             "u'", "v'", "Δ(u', v')", "a*", "b*", "L*", "Δa*", "Δb*", "ΔL*", "ΔEab*", "CR", "ΔCR(%)", "T%", "Scatter", "D", "W", 
             "Tr(ms)", "Tf(ms)", "RT(ms)", "G2G(ms)"]
)
# cell gap range
# +- 0.5 um, precise to 0.1 um
center_cell_gap = np.round(ref_cell_gap, decimals=1)
cell_gap_range = np.linspace(center_cell_gap-0.5, center_cell_gap+0.5, 11)

# for Eab
def F(X, opt):
    BLU = {
        "Xn": 95.04,
        "Yn": 100.00,
        "Zn": 108.86
    }
    result = 7.787 * X/BLU[opt] + 16/116
    result = result if X/BLU[opt] < 0.008856 else (X/BLU[opt]) ** (1/3)
    return result

# CR
X = [[ref_Vop, ref["cell gap(um)"][0]]]
ref_LC = ref["LC"][0]
ref_CR = ref["CR"][0]
ref_CR_index = ref_CR / (model["opt"][ref_LC]["T%_LR"].predict(X)[0]/float(prop[prop["LC"] == ref_LC]["Scatter index"])/ref["cell gap(um)"][0])
        
for LC in cond["LC"].unique():
    ne = float(prop[prop["LC"] == LC]["n_e"])
    no = float(prop[prop["LC"] == LC]["n_o"])
    scatter_index = float(prop[prop["LC"] == LC]["Scatter index"])
    for cell_gap in cell_gap_range:
        summary_table = summary_table.append({"LC": LC, "Gap(um)": cell_gap}, ignore_index=True)
        # rt
        X = [[ref_Vop, cell_gap]]
        X_minus = [[ref_Vop, cell_gap - 0.1]]
        Tr = model["rt"][LC]["Tr_LR"].predict(X)
        Tf = model["rt"][LC]["Tf_LR"].predict(X)
        RT = model["rt"][LC]["RT_LR"].predict(X)
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "RT(ms)"] = RT[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Tr(ms)"] = Tr[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Tf(ms)"] = Tf[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "G2G(ms)"] = ref["G2G(ms)"][0] / ref["RT(ms)"][0] * RT[0]

        # opt
        Wx = model["opt"][LC]["Wx_LR"].predict(X)
        Wx_gain = Wx - ref["Wx"][0]
        Wy = model["opt"][LC]["Wy_LR"].predict(X)
        Wy_gain = Wy - ref["Wy"][0]
        T = model["opt"][LC]["T%_LR"].predict(X)
        LCp = model["opt"][LC]["LC%_LR"].predict(X)
        Δnd = (ne - no) * cell_gap * 1000
 
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Wx"] = Wx[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Wx_gain"] = Wx_gain[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Wy"] = Wy[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Wy_gain"] = Wy_gain[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "T%"] = T[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "LC%"] = LCp[0] * 100
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Δnd(nm)"] = Δnd
        
        WX = model["opt"][LC]["WX_LR"].predict(X)
        WY = model["opt"][LC]["WY_LR"].predict(X)
        WZ = model["opt"][LC]["WZ_LR"].predict(X)

        Wx_minus = model["opt"][LC]["Wx_LR"].predict(X_minus)
        Wy_minus = model["opt"][LC]["Wy_LR"].predict(X_minus)
        WX_minus = model["opt"][LC]["WX_LR"].predict(X_minus)
        WY_minus = model["opt"][LC]["WY_LR"].predict(X_minus)
        WZ_minus = model["opt"][LC]["WZ_LR"].predict(X_minus)

        # another way to reproduce WX, WZ from xyY
#         WX = [Wx[0] * WY[0] / Wy[0]]
#         WZ = [(1 - Wx[0] - Wy[0]) * WY[0] / Wy[0]]
        
        F_X = F(WX[0], "Xn")
        F_Y = F(WY[0], "Yn")
        F_Z = F(WZ[0], "Zn")
        a_star = 500 * (F_X - F_Y)
        b_star = 200 * (F_Y - F_Z)
        L_star = 116 * F_Y - 16
        u_prime = 4 * Wx[0] / (-2 * Wx[0] + 12 * Wy[0] + 3)
        v_prime = 9 * Wy[0] / (-2 * Wx[0] + 12 * Wy[0] + 3)
        
        F_X_minus = F(WX_minus[0], "Xn")
        F_Y_minus = F(WY_minus[0], "Yn")
        F_Z_minus = F(WZ_minus[0], "Zn")
        a_star_minus = 500 * (F_X_minus - F_Y_minus)
        b_star_minus = 200 * (F_Y_minus - F_Z_minus)
        L_star_minus = 116 * F_Y_minus - 16
        Δa_star = a_star - a_star_minus
        Δb_star = b_star - b_star_minus
        ΔL_star = L_star - L_star_minus
        ΔE_ab_star = (Δa_star**2 + Δb_star**2 + ΔL_star**2)**(1/2)
        u_prime_minus = 4 * Wx_minus[0] / (-2 * Wx_minus[0] + 12 * Wy_minus[0] + 3)
        v_prime_minus = 9 * Wy_minus[0] / (-2 * Wx_minus[0] + 12 * Wy_minus[0] + 3)
        Δuv = ((u_prime - u_prime_minus)**2 + (v_prime - v_prime_minus)**2)**(1/2)
        
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "a*"] = a_star
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "b*"] = b_star
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "L*"] = L_star
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Δa*"] = Δa_star
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Δb*"] = Δb_star
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "ΔL*"] = ΔL_star
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "ΔEab*"] = ΔE_ab_star

        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "u'"] = u_prime
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "v'"] = v_prime
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Δ(u', v')"] = Δuv

        # V%
        V90 = model["opt"][LC]["Vop_LR"].predict([[90, cell_gap]])
        V95 = model["opt"][LC]["Vop_LR"].predict([[95, cell_gap]])
        V99 = model["opt"][LC]["Vop_LR"].predict([[99, cell_gap]])
        Vmax = model["opt"][LC]["Vop_LR"].predict([[100, cell_gap]])
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "V90"] = V90[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "V95"] = V95[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "V99"] = V99[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Vmax"] = Vmax[0]
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Vop(V)"] = ref_Vop
        
        # CR
        Scatter = scatter_index * cell_gap
        D = Scatter
        W = T[0]
        CR = W/D * ref_CR_index
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "D"] = D
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "W"] = W
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "Scatter"] = Scatter
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "CR"] = CR
        summary_table.loc[((summary_table["LC"] == LC) & (summary_table["Gap(um)"] == cell_gap)), "ΔCR(%)"] = (CR-ref_CR)/ref_CR * 100
        
        


In [224]:
prop

,LC,Vendor,measure T(°C),Tni(°C),Tcn(°C),design cell gap,n_e,n_o,ε_∥,ε_⊥,rotation viscosity (γ1)(mPa⋅s),K11(pN),K22(pN),K33(pN),Scatter index,RT index
0,LCT-15-1098,Merck,25,85,None,3.1,1.577,1.480,3.6,7.0,83,14.7,7.35,15.1,0.007101,11.292517
1,SLC19V33L00,SLC,25,80,None,3.1,1.603,1.488,3.9,8.1,97,17.0,8.50,15.0,0.009360,11.411765
2,SLC20V87L00,SLC,25,80,None,3.1,1.587,1.484,3.6,6.9,76,14.6,7.30,14.8,0.008179,10.410959
3,AV369-031-TA,HCCH,25,80,None,3.1,1.590,1.486,3.5,6.9,53,15.4,7.70,17.0,0.007656,6.883117
4,ZIX-7054XX,JNC,25,80,None,3.1,1.589,1.486,3.5,6.6,88,13.5,6.75,14.8,0.008586,13.037037


In [218]:
ref

,project,batch,LC,platform,cell gap(um),Vop(V),T%,Tr(ms),Tf(ms),RT(ms),G2G(ms),Wx,Wy,CR
0,1098-like-TR2,RD11001105,LCT-15-1098,xxxx,3.12,3.94,99,15.072,6.318,12.728,15.57,0.2588,0.272725,1410.8


In [228]:
summary_table.iloc[:, 15:]

,"Δ(u', v')",a*,b*,L*,Δa*,Δb*,ΔL*,ΔEab*,CR,ΔCR(%),T%,Scatter,D,W,Tr(ms),Tf(ms),RT(ms),G2G(ms)
0,0.003340,21.430769,-61.246373,159.540832,-2.933505,2.927958,6.469297,7.683111,1605.699493,13.814821,74.118436,0.018462,0.018462,74.118436,12.321531,7.526505,19.857796,24.291788
1,0.003328,19.126106,-58.391325,164.984861,-2.304663,2.855048,5.444030,6.565077,1562.388495,10.744861,74.893034,0.019172,0.019172,74.893034,12.890663,8.175750,21.067466,25.771563
2,0.003319,17.283952,-55.495771,169.567266,-1.842154,2.895555,4.582405,5.725050,1522.171139,7.894183,75.667632,0.019882,0.019882,75.667632,13.459795,8.824995,22.277137,27.251337
3,0.003311,15.797175,-52.477667,173.403497,-1.486777,3.018104,3.836231,5.102560,1484.727394,5.240104,76.442230,0.020592,0.020592,76.442230,14.028927,9.474239,23.486807,28.731112
4,0.003305,14.593839,-49.270910,176.577199,-1.203336,3.206757,3.173703,4.669443,1449.779899,2.762964,77.216828,0.021302,0.021302,77.216828,14.598059,10.123484,24.696478,30.210886
5,0.003300,13.624213,-45.816286,179.149874,-0.969626,3.454625,2.572675,4.415118,1417.087080,0.445639,77.991426,0.022012,0.022012,77.991426,15.167191,10.772728,25.906148,31.690661
6,0.003298,12.853316,-42.055371,181.166850,-0.770897,3.760914,2.016976,4.336698,1386.437563,-1.726853,78.766024,0.022722,0.022722,78.766024,15.736323,11.421973,27.115819,33.170435
7,0.003296,12.256457,-37.925661,182.661095,-0.596859,4.129710,1.494244,4.432101,1357.645593,-3.767678,79.540622,0.023432,0.023432,79.540622,16.305455,12.071217,28.325489,34.650209
8,0.003296,11.816481,-33.355829,183.655680,-0.439976,4.569831,0.994585,4.697461,1330.547267,-5.688456,80.315220,0.024142,0.024142,80.315220,16.874587,12.720462,29.535160,36.129984
9,0.003298,11.522071,-28.260266,184.165362,-0.294410,5.095563,0.509681,5.129446,1304.997418,-7.499474,81.089818,0.024852,0.024852,81.089818,17.443718,13.369707,30.744830,37.609758


In [2]:
# query_total_table.py
import sys
import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sqlalchemy as sql
import os

# params_str = {
#     "LC": sys.argv[1],
#     "cell_gap_lower": sys.argv[2],
#     "cell_gap_upper": sys.argv[3],
# }
# test input
params_str = {
    "LC": "LCT-15-1098,SLC19V33L00",
    "cell_gap_lower": "2.8,2.8",
    "cell_gap_upper": "3.4,3.4",
}

engine = sql.create_engine('sqlite:///./database/test.db', echo=False)

params = {}
for k, v in params_str.items():
    params[k] = v.split(",")

# print(os.getcwd())

result_df = pd.DataFrame()
# output = './public/tmp/'
output = './tmp/'
rnd_file_code = f"-{np.random.randint(0, 10000):04d}"
file_name = output + 'query-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime()) + rnd_file_code
for i in range(len(params["LC"])):
    print(params['LC'][i])
    tmp_df = pd.read_sql(f"SELECT * FROM summary WHERE LC == \"{params['LC'][i]}\" AND \"Gap(um)\" > {params['cell_gap_lower'][i]} AND \"Gap(um)\" < {params['cell_gap_upper'][i]}", engine)
    result_df = pd.concat([result_df, tmp_df], ignore_index=True)

result_df.to_excel(file_name + '.xlsx')

# print(params)
print(file_name + ".xlsx")
# print('python finished')

LCT-15-1098
SLC19V33L00
./tmp/query-2021-08-11-07-35-03-4018.xlsx


In [102]:
# reading property table test
# for now it's just messy trash...
pd.read_excel("old_ref/20200924_液晶data sheet (總整理) .xlsx",sheet_name="量產負型AAS",index_col=False)

,產品,*,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Mobile,Mobile.1,...,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,NB,Unnamed: 22,Unnamed: 23
0,Type,*,AAS,NaN,AAS,NaN,AAS,NaN,AAS,AAS,...,AAS,NaN,AAS,AAS,AAS,NaN,NaN,AAS,NaN,NaN
1,LC Name,*,LCT-13-1386,NaN,LCT-16-1381,NaN,LCT-15-1098,NaN,ZYH-7013XX,LCT-16-1228,...,LCT-16-1381,NaN,LCT-17-1336,LCT-19-580,SLC19V33L00,LCT-16-1200,ZIX-7244XX,LCT-19-1113,NaN,NaN
2,廠區,*,"南廠, T1",NaN,T3,NaN,T1,NaN,L6,T6,...,T3,NaN,T6/T3/T2,T6,NaN,NaN,NaN,NaN,NaN,NaN
3,廠商,NaN,Merck,NaN,Merck,NaN,Merck,NaN,JNC,Merck,...,Merck,NaN,Merck,Merck,SliChem,Merck,JNC,Merck,NaN,NaN
4,廠商量測LC溫度(℃),NaN,20,25.000000,20,25.000000,20,25.000000,25,20,...,20,25.000000,25,25,25,25,25,20,25.000000,30.000000
5,Transition Temp.,Tni(℃),84.5,NaN,84.9,84.900000,85.2,85.200000,80,78.2,...,84.9,84.900000,84.6,79.4,79.9,91.1,NaN,77.8,NaN,NaN
6,NaN,Tcn(℃),-30,-30.000000,NaN,NaN,-30,-30.000000,<-30,-30,...,-30,-30.000000,-30,-30,-30,-20,-15,NaN,NaN,NaN
7,cell Gap (um),NaN,NaN,3.100000,2.8,2.800000,3.1,3.100000,3.2,3.2,...,2.8,2.800000,2.8,3,2.8,3.65,3.65,3.2,3.200000,3.200000
8,d*△n (um),NaN,NaN,0.294810,0.29904,0.293720,0.30256,0.298220,0.32,0.32512,...,0.29904,0.293720,0.3234,0.3201,0.32228,0.309885,NaN,0.30848,0.302080,0.295680
9,Optical Anisotropy (589....,ne,1.5774,1.574100,1.5905,1.587200,1.5794,1.576900,1.586,1.5832,...,1.5905,1.587200,1.6015,1.5925,1.6028,1.563,NaN,1.5776,1.574300,1.571100
